<h1><center> 2. Patterns</center></h1>

## 2.1 Assertions 
should never be raised unless there is a bug in code.
Its execution:
<br>if __debug__: 
<br>&emsp;if not Condition: 
<br>&emsp;&emsp; AssertionError('msg') 
<br>asserts can be globaly disabled

In [1]:
assert 2+2 == 4, 'it s end of the world'

## 2.2 comma placement even at the end. 

In [2]:
# Be aware of python string concat:
['Alice'
'Bob']

['AliceBob']

## 2.3 with statement

In [3]:
with open('hello.txt', 'w') as f:
    f.write('hello world')

$execution:$
<br>f = open('hello.txt', 'w')
<br>try: 
<br>&emsp;f.write('hello world')
<br>finally: 
<br> &emsp;f.close()
<br>Try is important as if f.write fails, the file won't be closed. Leak!
<br>You can charge your own class with $with$ feature (see also @contextmanager decorator)

In [4]:
class Tab():
    def __init__(self):
        self._level = 0
        
    def __enter__(self):
        self._level +=1
        return self #object = __enter__(self)
    
    def __exit__(self, type_, val, tb):
        self._level -=1
        
    def print(self, text):
        print('    '*self._level + text)

In [5]:
with Tab() as tab:
    tab.print('enter 0 level')
    with tab:
        tab.print('enter 1 level')
    tab.print(' exit 0 level')

    enter 0 level
        enter 1 level
     exit 0 level


## 2.4 Underscores
1. _$var$: convention for internal use only (not imported with wildcard import *)
- $var$_ : convention to avoid naming conflict
-  _ : dummy OR result of the last expression in REPL session
-  __ $var$__ : special Python methods
-  __$var$: name mangling 
<br>
__$var$ is stored as <font color='blue'>_ClassName__var</font>  to avoid name conflicts for inherited classes. 
<br> but inside a class you can refer to simple self.__$var$. It would automatically call <font color='blue'>_ClassName__var</font>

In [6]:
class dunder():
    def __init__(self):
        self.__var = 'hello'
    def get_var(self):
        return self.__var
    
# dunder().__var    
"dunder object has no attribute __var"
dunder().get_var()

'hello'

## 2.5 String formatting

In [7]:
my_str = 'people'
my_flt = 2019.
'Welcome %s to %f' % (my_str, my_flt)                                 # old style
'Welcome {my_str} to {my_flt:f}'.format(my_str=my_str, my_flt=my_flt) # new style
f'Welcome {my_str} to {my_flt}'                                       # f-string

'Welcome people to 2019.0'

<h1><center> 3. Functions </center></h1>
Behavior in your program

## 3.1 First-Class functions. 
- assign them to variables
- store them in data structures
- pass as arguments to other functions
- return them as values from other functions

In [8]:
def shout(text):
    return text.upper() + '!'

cry = shout
del shout                              # no shout anymore
print('cry.__name__ = ', cry.__name__) # for debug
cry('hey')

cry.__name__ =  shout


'HEY!'

__High order functions__ accept other functions as arguments. Map() is classic example

In [9]:
list(map(cry, ['hello', 'ok', 'nice']))

['HELLO!', 'OK!', 'NICE!']

__Nested functions__ are functions defined in main function every time you call the last one. 

In [10]:
def speak_factory(volume):
    def low(text):
        return text.lower() + '...'
    def shout(text):
        return text.upper() + '!'
    if volume > 0.5:
        return shout
    else:
        return low

speak_factory returns the BEHAVIOUR (other function).
<br>
Nested function do not exist outside, but they can be returned. 


### Closures: Function that closure local state 
Above, we can redefine (with second argument) spreak_factory (volume, text) and the nested function without arguments: just low() and shout().
<br>
In this case the returned function is again nested but already with charged text argument in memory from parent function!
<br>
Speak_factory becomes less universal as we alreaddy charge the Local State to returned function. We closure the universality by precise local state.
<br>
charged_func = speak_factory ('I am local state', 0.7)
<br>
charged_func() &emsp;&emsp;&emsp; <-- simple call without arguments. They were already charged/closured

Other example:

In [11]:
def make_adder(n):
    def add(x):
        return x + n
    return add

We closure universal function add to specific function (ex. plus_n) by parameter n so it can add only +3 for any passing argument

In [12]:
plus_3 = make_adder(3)
plus_3(5)

8

We could do the same via classes: defining $__call__$ method that returns self.n + x
<br>
Class Adder ...(to define) ...
<br>
plus_3 = Adder(3)

## 3.2 Lambda is single-expression function

In [13]:
(lambda x, y: x+y) (5,3)

8

In [14]:
pairs = [(1,'c'), (2, 'a'), (3,'b')]
sorted(pairs, key = lambda x: x[1])

[(2, 'a'), (3, 'b'), (1, 'c')]

you can sort by any arbitrary rule. just precise it after lambda x: x*exp(x). F
<br>
p.s. see also .itemgetter() function. it's more concise

Lambdas may be used for nested functions

In [15]:
def make_adder(n):
    return lambda x: n + x
plus_3 = make_adder(3)
plus_3(4)

7

Do not abuse the use of lambda. Consider simple use only. (c) ZenPy: Readability counts 

## 3.4 * args & ** kwargs

In [16]:
def foo(x, *args, **kwargs):
    print(x, args, kwargs)
    args = args + ('extra', 666)
    kwargs['new'] = 'Im new'
    print(x, args, kwargs)
    
foo('hello', 1, 2, 3, key1 = 'value', key2 = 999)

hello (1, 2, 3) {'key1': 'value', 'key2': 999}
hello (1, 2, 3, 'extra', 666) {'key1': 'value', 'key2': 999, 'new': 'Im new'}


In [17]:
class ABCD:
    def __init__(self, a, b, c, d):
        self.A = a; self.B = b; self.C = c; self.D = d
        
class D4 (ABCD):
    def __init__(self, *args):
        super().__init__(*args)
        self.D = 4
        
D4(1,1,1,'dummy').D

4

if you add new arguments to ABCD class: __ init __ (self, a, b, c, d, e, f ..)
<br>
no need to change D4 class

### Unpacking

In [18]:
def print_vector(x,y,z):
    print(f'{x}, {y}, {z}')
    
print_vector(1,0,1)

list_vec = [1, 0, 1]
print_vector(*list_vec)

dict_vec = {'y': 0, 'x': 1, 'z': 1}
print_vector(*dict_vec)
print_vector(**dict_vec)

1, 0, 1
1, 0, 1
y, x, z
1, 0, 1


Dicts are __unordered__ => unpacking matches func args to dict values based on dict keys

## 3.3 Decorators
decorate or wrap another function. they can change or extend the behavior without modifying wrapped function itself

In [19]:
def shout(func):
    def wrapper():
        return func().upper() + '!'
    return wrapper

@shout
def hello():
    """hello doc"""
    return 'hello'

hello()

'HELLO!'

syntax: <font color='purple'>@decorator</font> <>
func = decorator(func) &emsp;&emsp;&emsp; <- func is overwritten
- decorated function is a new function 
- want to keep initial func?  decorate explicitely func2 = decorator(func)
- multi decorators from bottom to top

In [20]:
def traces(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f'TRACE: calling {func.__name__}() with {args}, {kwargs} \n'
              f'returned: {result}')
        return result
    return wrapper

In [21]:
@traces
def suma (a,b):
    return a+b

suma(3,4)

TRACE: calling suma() with (3, 4), {} 
returned: 7


7

Decorator hides some metadata from original function:

In [22]:
def ok():
    "say ok"
    return 'ok'
ok.__name__, ok.__doc__

('ok', 'say ok')

In [23]:
ok2 = shout(ok)
ok2.__name__, ok2.__doc__

('wrapper', None)

In [24]:
import functools

def shout(func):
    @functools.wraps(func)
    def wrapper():
        return func().upper() + '!'
    return wrapper

ok2 = shout(ok)
ok2.__name__, ok2.__doc__

('ok', 'say ok')